In [1]:
import numpy as np
import pandas as pd
import csv
import json
from sklearn import preprocessing

In [2]:
'''
ID - an Id that represents a (Shop, Item) tuple within the test set
shop_id - unique identifier of a shop
item_id - unique identifier of a product
item_category_id - unique identifier of item category
item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
item_price - current price of an item
date - date in format dd/mm/yyyy
date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
item_name - name of item
shop_name - name of shop
item_category_name - name of item category
'''

'\nID - an Id that represents a (Shop, Item) tuple within the test set\nshop_id - unique identifier of a shop\nitem_id - unique identifier of a product\nitem_category_id - unique identifier of item category\nitem_cnt_day - number of products sold. You are predicting a monthly amount of this measure\nitem_price - current price of an item\ndate - date in format dd/mm/yyyy\ndate_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33\nitem_name - name of item\nshop_name - name of shop\nitem_category_name - name of item category\n'

In [3]:
items = pd.read_csv("../data/items.csv")
item_cat = pd.read_csv("../data/item_categories.csv")
train = pd.read_csv("../data/sales_train.csv")
sample_submission = pd.read_csv("../data/sample_submission.csv")
shops = pd.read_csv("../data/shops.csv")
test = pd.read_csv("../data/test.csv")

In [4]:
print('items : ', len(items))
print('item_cat : ', len(item_cat))
print('train : ' , len(train))
print('test : ', len(test))
print('shops : ', len(shops))

items :  22170
item_cat :  84
train :  2935849
test :  214200
shops :  60


In [5]:
train['date'] = train['date'].str[3:]

In [6]:
train_groupby = train.groupby(['shop_id', 'item_id', 'date', 'item_price', 'date_block_num'])['item_cnt_day'].agg('sum').reset_index()

In [7]:
train = train_groupby

In [8]:
train_groupby_shop = train_groupby.groupby(['shop_id'])['item_cnt_day'].agg('sum').reset_index()
train_groupby_item = train_groupby.groupby(['item_id'])['item_cnt_day'].agg('sum').reset_index()
train_groupby_shop_and_item = train_groupby.groupby(['shop_id', 'item_id','date'])['item_cnt_day'].agg('sum').reset_index()

In [9]:
train_groupby_item

,item_id,item_cnt_day
0,0,1.0
1,1,6.0
2,2,2.0
3,3,2.0
4,4,1.0
...,...,...
21802,22165,2.0
21803,22166,348.0
21804,22167,1320.0
21805,22168,6.0


In [10]:
## train_groupby_item에서 item_cnt_day를 #2_amount로 이름 변경
train_groupby_shop = train_groupby_shop.rename(columns = {'item_cnt_day': '#2_amount'}, inplace = False)
train = pd.merge(train, train_groupby_shop, on='shop_id')

In [11]:
train['tmp'] = train['item_price']*train['item_cnt_day']
train_groupby_shop_price = train.groupby(['shop_id'])['tmp'].agg('sum').reset_index()
train_groupby_shop_price = train_groupby_shop_price.rename(columns = {'tmp': '#2_price'}, inplace = False)
train = pd.merge(train, train_groupby_shop_price, on ='shop_id')

In [12]:
train = pd.merge(train, items[['item_id', 'item_category_id']], on = 'item_id')

In [13]:
train_groupby_item_category = train.groupby(['item_category_id'])['item_cnt_day'].agg('sum').reset_index()

In [14]:
train_groupby_item_category = train_groupby_item_category.rename(columns = {'item_cnt_day': '#3_amount'}, inplace = False)

In [15]:
train = pd.merge(train, train_groupby_item_category , on='item_category_id')

In [16]:
train_groupby_item = train_groupby_item.rename(columns = {'item_cnt_day' : '#4_amount'}, inplace = False)

In [17]:
train = pd.merge(train, train_groupby_item , on='item_id')

In [18]:
train['month'] = train['date'].str[:2]

In [19]:
train['year'] = train['date'].str[3:]

In [20]:
def normalize_column(df, column_name):
    x = df[[column_name]]
    # Create x, where x the 'scores' column's values as floats
    x = x.values.astype(float)
    # Create a minimum and maximum processor object
    min_max_scaler = preprocessing.MinMaxScaler()
    # Create an object to transform the data to fit minmax processor
    x_scaled = min_max_scaler.fit_transform(x)
    # Run the normalizer on the dataframe
    df_normalized = pd.DataFrame(x_scaled, columns = [column_name])
    
    df[column_name] = df_normalized
    
    return df

In [21]:
#####test#####
test['month'] = 11
test['year'] = 2015
test['date_block_num'] =34

test = pd.merge(test,items, on = 'item_id')

test.drop('item_name', axis =1)

,ID,shop_id,item_id,month,year,date_block_num,item_category_id
0,0,5,5037,11,2015,34,19
1,5100,4,5037,11,2015,34,19
2,10200,6,5037,11,2015,34,19
3,15300,3,5037,11,2015,34,19
4,20400,2,5037,11,2015,34,19
...,...,...,...,...,...,...,...
214195,193799,46,969,11,2015,34,37
214196,198899,41,969,11,2015,34,37
214197,203999,44,969,11,2015,34,37
214198,209099,39,969,11,2015,34,37


In [22]:
test = pd.read_csv("test_mod1.csv")

test = test.drop("temp", axis = 1)
test = test.drop("item_name", axis = 1)

test['month'] = 11
test['year'] = 2015
test['date_block_num'] =34

# test = pd.merge(test,items, on = 'item_id')

# test.drop('item_name', axis =1)

# train_1 = train[train['date'] == '10.2015'][['shop_id','item_id', 'item_price']]

# df.join(other, lsuffix='_caller', rsuffix='_other')
# test2 = test.merge(train_1, how='left', on = ['item_id','shop_id'])
# test2

test2 = pd.merge(test, train_groupby_shop_price, on ='shop_id')
test2 = pd.merge(test2, train_groupby_shop, on='shop_id')


test2 = pd.merge(test2, train_groupby_item_category , on='item_category_id')
test2 = pd.merge(test2, train_groupby_item , on='item_id')

In [23]:
merged_data = pd.concat([train, test2], sort=False)

In [24]:
normalize_column(train, '#2_amount')
normalize_column(train, '#2_price')
normalize_column(train, '#3_amount')
normalize_column(train, '#4_amount')
train['month'] = train['month'].astype(int)
train['year'] = train['year'].astype(int)

In [25]:
train = merged_data.iloc[:1739022]
test = merged_data.iloc[1739022:]

In [26]:
test = test.drop("item_cnt_day", axis = 1)
test = test.drop("tmp", axis = 1)
test = test.drop("date", axis = 1)
test = test.drop("data_block_num", axis = 1)

In [27]:
test

,shop_id,item_id,item_price,date_block_num,#2_amount,#2_price,item_category_id,#3_amount,#4_amount,month,year,ID
0,5,5037,749.0,34,42762.0,3.816043e+07,19,254887.0,1270.0,11,2015,0.0
1,4,5037,749.0,34,43942.0,4.053965e+07,19,254887.0,1270.0,11,2015,5100.0
2,6,5037,1499.0,34,100489.0,9.806745e+07,19,254887.0,1270.0,11,2015,10200.0
3,3,5037,1499.0,34,28355.0,3.014085e+07,19,254887.0,1270.0,11,2015,15300.0
4,2,5037,749.0,34,30620.0,4.404964e+07,19,254887.0,1270.0,11,2015,20400.0
...,...,...,...,...,...,...,...,...,...,...,...,...
198949,46,180,80.0,34,78990.0,7.418932e+07,44,252.0,1.0,11,2015,193261.0
198950,41,180,80.0,34,49324.0,4.609299e+07,44,252.0,1.0,11,2015,198361.0
198951,44,180,80.0,34,44938.0,3.420403e+07,44,252.0,1.0,11,2015,203461.0
198952,39,180,80.0,34,16617.0,1.809289e+07,44,252.0,1.0,11,2015,208561.0


# 6.1

In [274]:
train['item_price_avg'] = train['item_price'].mean()

# 6.2

In [275]:
out = []
unique_cat = train['item_category_id'].unique()
for cat in unique_cat:
    same_cat_diff_shop = train[train['item_category_id']==cat]
    same_shop = same_cat_diff_shop.drop_duplicates(subset='shop_id', keep="last") #unique shops
    norm=(same_shop['item_price']-same_shop['item_price'].mean())/same_shop['item_price'].std()
    same_shop['item_cat_price_norm'] = norm
    arr = np.array(same_shop)[:,[2,8, -1]]
    out.append(arr)
    #out.append(same_shop['date','item_category_id' ,'item_cat_price_norm'])
    #train = train.merge(same_shop, on = list(train.columns), how = 'left')

/opt/anaconda3/envs/jaeyoung/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [276]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,#2_amount,tmp,#2_price,item_category_id,#3_amount,#4_amount,month,year,item_price_avg,item_cat_price_norm
0,01.2013,0,59,22154,999.0,1.0,0.156751,999.0,0.190977,37,0.32055,0.000373,1,2013,890.853233,NaN
1,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,0.32055,0.000373,1,2013,890.853233,NaN
2,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,0.32055,0.000373,1,2013,890.853233,NaN
3,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,0.32055,0.000373,1,2013,890.853233,NaN
4,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,0.32055,0.000373,1,2013,890.853233,NaN


In [277]:
out = np.array(out)
v = np.vstack(out)
df = pd.DataFrame(v, columns = ['shop_id', 'item_category_id', 'item_cat_price_norm'])
df = df.fillna(0)
df_train = train.merge(df, on = ['shop_id', 'item_category_id'], how = 'left')
# train = df_train.drop(['item_cat_price_norm_x'], axis = 1)
train = df_train

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [ ]:
train.head()

# 1

In [112]:
train = pd.concat([train,pd.get_dummies(train['shop_id'], prefix='shop_id')],axis=1)

In [113]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,#2_amount,tmp,#2_price,item_category_id,...,shop_id_50,shop_id_51,shop_id_52,shop_id_53,shop_id_54,shop_id_55,shop_id_56,shop_id_57,shop_id_58,shop_id_59
0,01.2013,0,59,22154,999.0,1.0,0.156751,999.0,0.190977,37,...,0,0,0,0,0,0,0,0,0,1
1,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0,0,0
2,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0,0,0
3,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0,0,0
4,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,10.2015,33,55,13093,250.0,1.0,0.203120,250.0,0.210418,36,...,0,0,0,0,0,1,0,0,0,0
2935845,09.2015,32,55,13091,1000.0,1.0,0.203120,1000.0,0.210418,36,...,0,0,0,0,0,1,0,0,0,0
2935846,09.2015,32,55,13094,2500.0,1.0,0.203120,2500.0,0.210418,36,...,0,0,0,0,0,1,0,0,0,0
2935847,09.2015,32,55,13094,2500.0,2.0,0.203120,5000.0,0.210418,36,...,0,0,0,0,0,1,0,0,0,0


## 8

In [114]:
lag2015 = train[train['date'] == '10.2015']['item_cnt_day']
lag2013 = train[train['date'] == '11.2013']['item_cnt_day']
lag2014 = train[train['date'] == '11.2014']['item_cnt_day']

In [115]:
lag2013.shape, lag2014.shape, lag2015.shape

((96736,), (86428,), (53514,))

In [116]:
lag2013_ = lag2013.to_frame()
lag2014_ = lag2014.to_frame()
lag2015_ = lag2015.to_frame()

temp2 = pd.concat([lag2013_, lag2014_,lag2015_], axis=1).fillna(0)
temp2.columns=['2013.11','2014.11','2015.10']
temp2

,2013.11,2014.11,2015.10
9,0.0,0.0,1.0
14,0.0,0.0,1.0
25,0.0,0.0,1.0
31,0.0,0.0,1.0
45,0.0,0.0,1.0
...,...,...,...
2935841,0.0,0.0,1.0
2935842,0.0,0.0,1.0
2935843,0.0,0.0,1.0
2935844,0.0,0.0,1.0


In [117]:
train = pd.concat([train, temp2], axis=1).fillna(0)
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,#2_amount,tmp,#2_price,item_category_id,...,shop_id_53,shop_id_54,shop_id_55,shop_id_56,shop_id_57,shop_id_58,shop_id_59,2013.11,2014.11,2015.10
0,01.2013,0,59,22154,999.0,1.0,0.156751,999.0,0.190977,37,...,0,0,0,0,0,0,1,0.0,0.0,0.0
1,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0.0,0.0,0.0
2,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0.0,0.0,0.0
3,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0.0,0.0,0.0
4,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,10.2015,33,55,13093,250.0,1.0,0.203120,250.0,0.210418,36,...,0,0,1,0,0,0,0,0.0,0.0,1.0
2935845,09.2015,32,55,13091,1000.0,1.0,0.203120,1000.0,0.210418,36,...,0,0,1,0,0,0,0,0.0,0.0,0.0
2935846,09.2015,32,55,13094,2500.0,1.0,0.203120,2500.0,0.210418,36,...,0,0,1,0,0,0,0,0.0,0.0,0.0
2935847,09.2015,32,55,13094,2500.0,2.0,0.203120,5000.0,0.210418,36,...,0,0,1,0,0,0,0,0.0,0.0,0.0


## 9 

In [118]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [119]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,#2_amount,tmp,#2_price,item_category_id,...,shop_id_53,shop_id_54,shop_id_55,shop_id_56,shop_id_57,shop_id_58,shop_id_59,2013.11,2014.11,2015.10
0,01.2013,0,59,22154,999.0,1.0,0.156751,999.0,0.190977,37,...,0,0,0,0,0,0,1,0.0,0.0,0.0
1,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0.0,0.0,0.0
2,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0.0,0.0,0.0
3,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0.0,0.0,0.0
4,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0.0,0.0,0.0


In [120]:
from sklearn.feature_extraction.text import CountVectorizer

item_name_list = items['item_name']

vect = CountVectorizer(analyzer='word', max_features=20)
bow_feature = vect.fit_transform(item_name_list).toarray()
bow_feature

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [121]:
type(bow_feature)

numpy.ndarray

In [122]:
bow_df = pd.DataFrame(bow_feature)
bow_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22165,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1
22166,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
22167,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22168,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [123]:
item_ids = train['item_id']
item_ids

0          22154
1          22154
2          22154
3          22154
4          22154
           ...  
2935844    13093
2935845    13091
2935846    13094
2935847    13094
2935848    13092
Name: item_id, Length: 2935849, dtype: int64

In [124]:
temp = bow_df.iloc[item_ids].reset_index(drop=True)
temp

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2935845,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2935846,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2935847,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1


In [125]:
train_ = pd.concat([train, temp], axis=1)
train_

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,#2_amount,tmp,#2_price,item_category_id,...,10,11,12,13,14,15,16,17,18,19
0,01.2013,0,59,22154,999.0,1.0,0.156751,999.0,0.190977,37,...,0,0,0,0,0,0,0,0,0,0
1,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0,0,0
2,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0,0,0
3,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0,0,0
4,01.2013,0,25,22154,999.0,1.0,0.778200,999.0,0.920216,37,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,10.2015,33,55,13093,250.0,1.0,0.203120,250.0,0.210418,36,...,0,0,1,0,0,0,0,0,0,1
2935845,09.2015,32,55,13091,1000.0,1.0,0.203120,1000.0,0.210418,36,...,0,0,1,0,0,0,0,0,0,1
2935846,09.2015,32,55,13094,2500.0,1.0,0.203120,2500.0,0.210418,36,...,0,0,1,0,0,0,0,0,0,1
2935847,09.2015,32,55,13094,2500.0,2.0,0.203120,5000.0,0.210418,36,...,0,0,1,0,0,0,0,0,0,1


## TEST!!

In [69]:
test = pd.read_csv("test_mod1.csv")

test = test.drop("temp", axis = 1)
test = test.drop("item_name", axis = 1)

test['month'] = 11
test['year'] = 2015
test['data_block_num'] =34

# test = pd.merge(test,items, on = 'item_id')

# test.drop('item_name', axis =1)

# train_1 = train[train['date'] == '10.2015'][['shop_id','item_id', 'item_price']]

# df.join(other, lsuffix='_caller', rsuffix='_other')
# test2 = test.merge(train_1, how='left', on = ['item_id','shop_id'])
# test2

test2 = pd.merge(test, train_groupby_shop_price, on ='shop_id')
test2 = pd.merge(test2, train_groupby_shop, on='shop_id')


test2 = pd.merge(test2, train_groupby_item_category , on='item_category_id')
test2 = pd.merge(test2, train_groupby_item , on='item_id')

In [73]:
test2

,ID,shop_id,item_id,month,year,data_block_num,item_category_id,item_price,#2_price,#2_amount,#3_amount,#4_amount
0,0,5,5037,11,2015,34,19,749,3.816043e+07,42762.0,254887.0,1270.0
1,5100,4,5037,11,2015,34,19,749,4.053965e+07,43942.0,254887.0,1270.0
2,10200,6,5037,11,2015,34,19,1499,9.806745e+07,100489.0,254887.0,1270.0
3,15300,3,5037,11,2015,34,19,1499,3.014085e+07,28355.0,254887.0,1270.0
4,20400,2,5037,11,2015,34,19,749,4.404964e+07,30620.0,254887.0,1270.0
...,...,...,...,...,...,...,...,...,...,...,...,...
198949,193261,46,180,11,2015,34,44,80,7.418932e+07,78990.0,252.0,1.0
198950,198361,41,180,11,2015,34,44,80,4.609299e+07,49324.0,252.0,1.0
198951,203461,44,180,11,2015,34,44,80,3.420403e+07,44938.0,252.0,1.0
198952,208561,39,180,11,2015,34,44,80,1.809289e+07,16617.0,252.0,1.0


In [74]:
normalize_column(test2, '#2_amount')
normalize_column(test2, '#2_price')
normalize_column(test2, '#3_amount')
normalize_column(test2, '#4_amount')

##test랑 train이랑 합쳐서 

,ID,shop_id,item_id,month,year,data_block_num,item_category_id,item_price,#2_price,#2_amount,#3_amount,#4_amount
0,0,5,5037,11,2015,34,19,749,0.160888,0.136680,0.401901,0.006763
1,5100,4,5037,11,2015,34,19,749,0.171019,0.140481,0.401901,0.006763
2,10200,6,5037,11,2015,34,19,1499,0.415985,0.322628,0.401901,0.006763
3,15300,3,5037,11,2015,34,19,1499,0.126738,0.090273,0.401901,0.006763
4,20400,2,5037,11,2015,34,19,749,0.185965,0.097569,0.401901,0.006763
...,...,...,...,...,...,...,...,...,...,...,...,...
198949,193261,46,180,11,2015,34,44,80,0.314307,0.253377,0.000363,0.000000
198950,198361,41,180,11,2015,34,44,80,0.194666,0.157818,0.000363,0.000000
198951,203461,44,180,11,2015,34,44,80,0.144040,0.143690,0.000363,0.000000
198952,208561,39,180,11,2015,34,44,80,0.075435,0.052463,0.000363,0.000000


In [127]:
train = train.drop('tmp', axis =1)

In [139]:
train_mod.to_csv('train_mod.csv', header = True, index = False)

In [133]:
test.to_csv('test_mod.csv', header = True, index = False)

In [137]:
train_mod = train[['shop_id', 'item_id', 'month', 'year', 'item_price']]

In [75]:
test2

,ID,shop_id,item_id,month,year,data_block_num,item_category_id,item_price,#2_price,#2_amount,#3_amount,#4_amount
0,0,5,5037,11,2015,34,19,749,0.160888,0.136680,0.401901,0.006763
1,5100,4,5037,11,2015,34,19,749,0.171019,0.140481,0.401901,0.006763
2,10200,6,5037,11,2015,34,19,1499,0.415985,0.322628,0.401901,0.006763
3,15300,3,5037,11,2015,34,19,1499,0.126738,0.090273,0.401901,0.006763
4,20400,2,5037,11,2015,34,19,749,0.185965,0.097569,0.401901,0.006763
...,...,...,...,...,...,...,...,...,...,...,...,...
198949,193261,46,180,11,2015,34,44,80,0.314307,0.253377,0.000363,0.000000
198950,198361,41,180,11,2015,34,44,80,0.194666,0.157818,0.000363,0.000000
198951,203461,44,180,11,2015,34,44,80,0.144040,0.143690,0.000363,0.000000
198952,208561,39,180,11,2015,34,44,80,0.075435,0.052463,0.000363,0.000000
